In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!pip install --no-deps seqeval[gpu]
!pip install pytorch_pretrained_bert
!pip install seqeval
!python -m spacy download en_core_web_lg

  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7423 sha256=71922eb5b7d2f15211666586325deac9b0e33ed4a987d9fa4672b698e16fa7f8
  Stored in directory: /home/ahmed/.cache/pip/wheels/04/bf/20/90daf50b5a8173fc6ee715e6ebb0a16202cc43df471e323e88
Successfully built seqeval
     |████████████████████████████████| 123 kB 179 kB/s eta 0:00:01
     |                                | 2.6 MB 37 kB/s eta 6:06:550ERROR: Exception:
Traceback (most recent call last):
  File "/home/ahmed/miniconda3/envs/work/lib/python3.8/site-packages/pip/_vendor/urllib3/contrib/pyopenssl.py", line 313, in recv_into
    return self.connection.recv_into(*args, **kwargs)
  File "/home/ahmed/miniconda3/envs/work/lib/python3.8/site-packages/OpenSSL/SSL.py", line 1840, in recv_into
    self._raise_ssl_error(self._ssl, result)
  File "/home/ahmed/miniconda3/envs/work/lib/python3.8/site-packages/OpenSSL/SSL.py", line 1646, in _raise_ssl_error
    raise WantReadError()
OpenSSL.SSL.WantReadError

During

In [ ]:
from tqdm import tqdm,trange
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
from spacy.gold import biluo_tags_from_offsets

import pandas as pd
import math
import numpy as np
import torch.nn.functional as F
import torch
import os
import json
import random
import logging
import re
import torch
import nltk
import spacy
import string

spacy.cli.download('en_core_web_lg')
nlp = spacy.load("en_core_web_lg")
nltk.download('stopwords')
nltk.download('wordnet')

In [0]:
prefixes = ('\\n', ) + nlp.Defaults.prefixes
prefix_regex = spacy.util.compile_prefix_regex(prefixes)
nlp.tokenizer.prefix_search = prefix_regex.search

In [0]:
entity_dict = {
    
    'Address':'Address',
    'Can Relocate to':'CRT',
    'Certifications':'Certifications',
    'College':'College',
    'College Name':'College',
    'Companies worked at':'Companies',
    'Degree':'Degree',
    'Designation':'Designation',
    'Email Address':'Email',
    'Graduation Year':'Graduation',
    'Links':'Links',
    'Location':'Location',
    'Name':'Name',
    'Relocate to':'CRT',
    'Rewards and Achievements':'Achievements',
    'Skills':'Skills',
    'UNKNOWN':'UNKNOWN',
    'University':'University',
    'Years of Experience':'YOE',
    'des':'Designation',
    'projects':'projects',
    'state':'state',
    'training':'training',
    'links':'links',
    'abc':'UNKNOWN'

}


In [0]:
def mergeIntervals(intervals):
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []

    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            if higher[0] <= lower[1]:
                if lower[2] is higher[2]:
                    upper_bound = max(lower[1], higher[1])
                    merged[-1] = (lower[0], upper_bound, lower[2])
                else:
                    if lower[1] > higher[1]:
                        merged[-1] = lower
                    else:
                        merged[-1] = (lower[0], higher[1], higher[2])
            else:
                merged.append(higher)

    return merged

In [0]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines = []
        with open(dataturks_JSON_FilePath, 'r', encoding="utf8") as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            if data['annotation'] is not None:
                for annotation in data['annotation']:
                    # only a single point in text annotation.
                    point = annotation['points'][0]
                    labels = annotation['label']
                    # handle both list of labels or a single label.
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        # dataturks indices are both inclusive [start, end]
                        # but spacy is not [start, end)
                        entities.append((
                            point['start'],
                            point['end'] + 1,
                            label
                        ))

            training_data.append((text, {"entities": mergeIntervals(entities)}))
        return training_data
    except Exception:
        logging.exception("Unable to process " + dataturks_JSON_FilePath)
        return None

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data ,ents= [],[]
    ret=pd.DataFrame()
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append((valid_start, valid_end, entity_dict[label]))
        cleaned_data.append(text)
        ents.append(valid_entities)
    ret['content']=cleaned_data
    ret['entities']=ents
    return ret

In [0]:
df = trim_entity_spans(convert_dataturks_to_spacy("traindata.json"))

In [0]:
df.head()

,content,entities
0,Afreen Jamadar\nActive member of IIIT Committe...,"[(0, 14, Name), (62, 68, Location), (104, 148,..."
1,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[(0, 12, Name), (13, 51, Designation), (54, 60..."
2,Anvitha Rao\nAutomation developer\n\n- Email m...,"[(0, 11, Name), (12, 32, Designation), (56, 97..."
3,arjun ks\nSenior Program coordinator - oracle ...,"[(0, 8, Name), (9, 35, Designation), (38, 58, ..."
4,"Arun Elumalai\nQA Tester\n\nChennai, Tamil Nad...","[(0, 13, Name), (14, 23, Designation), (25, 32..."


In [0]:
def get_train_data(df):
    tags = []
    sentences = []

    for i in range(len(df)):
        text = df['content'][i]
        entities = df['entities'][i]
    
        doc = nlp(text)
    
        tag = biluo_tags_from_offsets(doc, entities)
        tmp = pd.DataFrame([list(doc), tag]).T
        loc = []
        for i in range(len(tmp)):
            if tmp[0][i].text is '.' and tmp[1][i] is 'O':
                loc.append(i)
        loc.append(len(doc))
    
        last = 0
        data = []
        for pos in loc:
            data.append([list(doc)[last:pos], tag[last:pos]])
            last = pos
    
        for d in data:
            tag = ['O' if t is '-' else t for t in d[1]]
            if len(set(tag)) > 1:
                sentences.append(d[0])
                tags.append(tag)
    
    return sentences, tags

In [0]:
sentences, tags = get_train_data(df)
print(len(sentences), len(tags))

3164 3164


In [0]:
tag_vals = set(['X', '[CLS]', '[SEP]'])
for i in range(len(tags)):
    tag_vals = tag_vals.union(tags[i])
print(tag_vals)

{'B-Designation', 'B-Achievements', 'L-Email', 'I-CRT', 'I-Skills', 'U-links', 'B-training', 'U-UNKNOWN', 'B-Links', 'I-UNKNOWN', 'U-state', 'L-Links', 'L-Designation', 'I-YOE', 'I-College', 'I-Graduation', 'L-Degree', 'U-Address', 'U-College', 'B-UNKNOWN', 'U-Location', 'L-Graduation', 'L-Address', 'L-YOE', 'X', 'B-University', 'U-Links', 'B-Degree', 'B-Address', 'U-Skills', 'U-Name', 'L-Achievements', '[CLS]', 'B-Graduation', 'U-Certifications', 'L-Location', 'B-YOE', 'I-projects', 'I-Email', 'U-Designation', 'I-Achievements', 'B-College', 'B-Location', 'B-Email', 'U-Degree', '[SEP]', 'L-Companies', 'L-Name', 'O', 'L-training', 'B-Name', 'B-Skills', 'L-state', 'B-projects', 'I-Companies', 'B-Certifications', 'L-UNKNOWN', 'L-University', 'U-Email', 'I-Name', 'I-Certifications', 'I-Location', 'I-Designation', 'I-training', 'L-Certifications', 'B-CRT', 'I-Address', 'I-Degree', 'L-projects', 'U-CRT', 'B-state', 'L-Skills', 'I-Links', 'L-CRT', 'B-Companies', 'U-YOE', 'U-Companies', 'I-Uni

In [0]:
tag2idx = {t: i for i, t in enumerate(tag_vals)}
tag2idx

{'B-Achievements': 1,
 'B-Address': 28,
 'B-CRT': 65,
 'B-Certifications': 55,
 'B-College': 41,
 'B-Companies': 74,
 'B-Degree': 27,
 'B-Designation': 0,
 'B-Email': 43,
 'B-Graduation': 33,
 'B-Links': 8,
 'B-Location': 42,
 'B-Name': 50,
 'B-Skills': 51,
 'B-UNKNOWN': 19,
 'B-University': 25,
 'B-YOE': 36,
 'B-projects': 53,
 'B-state': 70,
 'B-training': 6,
 'I-Achievements': 40,
 'I-Address': 66,
 'I-CRT': 3,
 'I-Certifications': 60,
 'I-College': 14,
 'I-Companies': 54,
 'I-Degree': 67,
 'I-Designation': 62,
 'I-Email': 38,
 'I-Graduation': 15,
 'I-Links': 72,
 'I-Location': 61,
 'I-Name': 59,
 'I-Skills': 4,
 'I-UNKNOWN': 9,
 'I-University': 77,
 'I-YOE': 13,
 'I-projects': 37,
 'I-training': 63,
 'L-Achievements': 31,
 'L-Address': 22,
 'L-CRT': 73,
 'L-Certifications': 64,
 'L-College': 78,
 'L-Companies': 46,
 'L-Degree': 16,
 'L-Designation': 12,
 'L-Email': 2,
 'L-Graduation': 21,
 'L-Links': 11,
 'L-Location': 35,
 'L-Name': 47,
 'L-Skills': 71,
 'L-UNKNOWN': 56,
 'L-Unive

In [0]:
res_idx_tag={'B-Achievements': 'Achievements',
 'B-Address': 'Address','B-CRT': 'CRT','B-Certifications': 'Certifications',
 'B-College': 'College', 'B-Companies': 'Companies','B-Degree': 'Degree',
 'B-Designation': 'Designation','B-Email': 'Email',
 'B-Graduation': 'Graduation','B-Links': 'Links','B-Location': 'Location',
 'B-Name': 'Name','B-Skills': 'Skills', 'B-UNKNOWN': 'UNKNOWN',
 'B-University': 'University', 'B-YOE': 'YOE', 'B-projects': 'Projects',
 'B-state': 'State', 'B-training': 'Training', 'I-Achievements': 'Achievements',
 'I-Address': 'Address', 'I-CRT': 'CRT', 'I-Certifications': 'Certifications',
 'I-College': 'College','I-Companies': 'Companies', 'I-Degree': 'Degree', 'I-Designation': 'Designation',
 'I-Email': 'Email', 'I-Graduation': 'Graduation', 'I-Links': 'Links',
 'I-Location': 'Location','I-Name': 'Name',
 'I-Skills': 'Skills','I-UNKNOWN': 'UNKNOWN',
 'I-University': 'University','I-YOE': 'YOE','I-projects': 'Projects',
 'I-training': 'Training', 'L-Achievements': 'Achievements',
 'L-Address': 'Address','L-CRT': 'CRT', 'L-Certifications': 'Certifications',
 'L-College': 'College', 'L-Companies':'Companies', 'L-Degree': 'Degree',
 'L-Designation': 'Designation','L-Email': 'Email','L-Graduation': 'Graduation','L-Links': 'Links',
 'L-Location': 'Location','L-Name':'Name','L-Skills': 'Skills', 'L-UNKNOWN': 'UNKNOWN',
 'L-University': 'University','L-YOE': 'YOE','L-projects': 'Projects','L-state': 'State',
 'L-training': 'Training', 'O': 'O','U-Address': 'Address','U-CRT': 'CRT','U-Certifications': 'Certifications',
 'U-College': 'College','U-Companies': 'Companies','U-Degree': 'Degree','U-Designation': 'Designation',
 'U-Email': 'Email','U-Graduation': 'Graduation','U-Links': 'Links','U-Location': 'Location','U-Name': 'Name',
 'U-Skills': 'Skills','U-UNKNOWN': 'UNKNOWN','U-YOE': 'YOE','U-links': 'Links','U-state': 'State',
 'X': 'X',
 '[CLS]': '[CLS]',
 '[SEP]': '[SEP]'}

In [0]:
idx2tag = {tag2idx[key] : key for key in tag2idx.keys()}


In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

100%|██████████| 213450/213450 [00:00<00:00, 1097315.15B/s]


In [0]:
def get_tokenized_train_data(sentences, tags):

    tokenized_texts = []
    word_piece_labels = []

    for word_list, label in zip(sentences, tags):
    
        # Add [CLS] at the front
        temp_lable = ['[CLS]']
        temp_token = ['[CLS]']
    
        for word, lab in zip(word_list, label):
            token_list = tokenizer.tokenize(word.text)
            for m, token in enumerate(token_list):
                temp_token.append(token)
                if m == 0:
                    temp_lable.append(lab)
                else:
                    temp_lable.append('X')  
                
        # Add [SEP] at the end
        temp_lable.append('[SEP]')
        temp_token.append('[SEP]')
    
        tokenized_texts.append(temp_token)
        word_piece_labels.append(temp_lable)
    
    return tokenized_texts, word_piece_labels

In [0]:
tokenized_texts, word_piece_labels = get_tokenized_train_data(sentences, tags)

In [0]:
MAX_LEN = 512
bs= 4

input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels], maxlen=MAX_LEN, value=tag2idx["O"], 
                     padding="post", dtype="long", truncating="post")

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (679 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (977 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (567 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (589 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1054 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length 

In [0]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [0]:
tr_inputs, val_inputs, tr_tags, val_tags, tr_masks, val_masks = train_test_split(input_ids, tags, attention_masks, random_state=2020, 
                                                                                 test_size=0.2)
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)


train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [0]:
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(tag2idx)).cuda()

100%|██████████| 404400730/404400730 [00:12<00:00, 31332469.46B/s]


In [0]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [0]:
epochs = 30
max_grad_norm = 1.0
mnloss=np.inf

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    if (tr_loss/nb_tr_steps) < mnloss:
        mnloss=(tr_loss/nb_tr_steps)
        torch.save(model.state_dict(),'/content/drive/My Drive/ner_resume_2.bin')

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Train loss: 0.5069324630555383


Epoch:   3%|▎         | 1/30 [02:57<1:25:48, 177.53s/it]

Train loss: 0.25183919164842056


Epoch:   7%|▋         | 2/30 [05:54<1:22:49, 177.50s/it]

...

Epoch:  93%|█████████▎| 28/30 [1:22:11<05:50, 175.25s/it]

Train loss: 0.014638689783605744
Train loss: 0.013503240860466118


Epoch:  97%|█████████▋| 29/30 [1:25:06<02:55, 175.30s/it]

Train loss: 0.01300073541456423


Epoch: 100%|██████████| 30/30 [1:28:02<00:00, 176.09s/it]


In [0]:
model.eval()

y_true = []
y_pred = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch

    with torch.no_grad():
        logits = model(input_ids, token_type_ids=None, attention_mask=input_mask,)

    logits = logits.detach().cpu().numpy()
    logits = [list(p) for p in np.argmax(logits, axis=2)]
    
    label_ids = label_ids.to('cpu').numpy()
    input_mask = input_mask.to('cpu').numpy()
    
    for i,mask in enumerate(input_mask):
        temp_1 = [] # Real one
        temp_2 = [] # Predict one
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if idx2tag[label_ids[i][j]] != "X" and idx2tag[label_ids[i][j]] != "[CLS]" and idx2tag[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(idx2tag[label_ids[i][j]])
                    temp_2.append(idx2tag[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)
    
print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

print(classification_report(y_true, y_pred,digits=4))

f1 socre: 0.732066
Accuracy score: 0.920459
              precision    recall  f1-score   support

 Designation     0.7252    0.8034    0.7623       473
        Name     0.9477    0.9797    0.9635       148
   Companies     0.7774    0.8136    0.7951       515
       Email     0.8647    0.8647    0.8647       133
  Graduation     0.7345    0.7477    0.7411       111
         YOE     0.4727    0.4522    0.4622       115
      Degree     0.7327    0.8196    0.7737       194
      Skills     0.6185    0.5911    0.6045       384
    Location     0.7760    0.8267    0.8006       352
     College     0.7043    0.8060    0.7517       201
         CRT     0.5000    0.1739    0.2581        23
       Links     0.2222    0.1176    0.1538        17
Achievements     0.1429    0.1250    0.1333         8
       state     0.5000    0.2000    0.2857         5
    projects     0.0000    0.0000    0.0000         2
     Address     0.0000    0.0000    0.0000         1
     UNKNOWN     0.0000    0.0000    

In [0]:
!pip install pdfminer.six

In [0]:
!pip install pytorch_pretrained_bert
from tqdm import tqdm,trange
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from spacy.gold import biluo_tags_from_offsets

import pandas as pd
import math
import numpy as np
import torch.nn.functional as F
import torch
import os
import json
import random
import logging
import re
import torch
import nltk
import spacy
import string

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
tag2idx={'B-Achievements': 1,'B-Address': 28,'B-CRT': 65,'B-Certifications': 55,'B-College': 41,'B-Companies': 74,
                      'B-Degree': 27,'B-Designation': 0,'B-Email': 43, 'B-Graduation': 33, 'B-Links': 8, 'B-Location': 42,
                      'B-Name': 50,'B-Skills': 51,'B-UNKNOWN': 19,'B-University': 25, 'B-YOE': 36, 'B-projects': 53,
                      'B-state': 70, 'B-training': 6,'I-Achievements': 40,'I-Address': 66, 'I-CRT': 3, 'I-Certifications': 60,
                      'I-College': 14, 'I-Companies': 54, 'I-Degree': 67, 'I-Designation': 62,'I-Email': 38,'I-Graduation': 15,'I-Links': 72,
                      'I-Location': 61,'I-Name': 59,'I-Skills': 4,'I-UNKNOWN': 9,'I-University': 77,'I-YOE': 13,'I-projects': 37,'I-training': 63,
                      'L-Achievements': 31,'L-Address': 22,'L-CRT': 73,'L-Certifications': 64,'L-College': 78,'L-Companies': 46,'L-Degree': 16,
                      'L-Designation': 12,'L-Email': 2,'L-Graduation': 21,'L-Links': 11,'L-Location': 35, 'L-Name': 47, 'L-Skills': 71,
                      'L-UNKNOWN': 56,'L-University': 57, 'L-YOE': 23, 'L-projects': 68, 'L-state': 52, 'L-training': 49,'O': 48,
                      'U-Address': 17, 'U-CRT': 69, 'U-Certifications': 34, 'U-College': 18, 'U-Companies': 76, 'U-Degree': 44, 'U-Designation': 39,
                      'U-Email': 58, 'U-Graduation': 79, 'U-Links': 26, 'U-Location': 20, 'U-Name': 30,
                     'U-Skills': 29,'U-UNKNOWN': 7,'U-YOE': 75, 'U-links': 5, 'U-state': 10, 'X': 24, '[CLS]': 32, '[SEP]': 45}
 
idx2tag = {tag2idx[key] : key for key in tag2idx.keys()}

res_idx_tag={'B-Achievements': 'Achievements', 'B-Address': 'Address','B-CRT': 'CRT','B-Certifications': 'Certifications',
                'B-College': 'College', 'B-Companies': 'Companies','B-Degree': 'Degree', 'B-Designation': 'Designation','B-Email': 'Email',
  '              B-Graduation': 'Graduation','B-Links': 'Links','B-Location': 'Location', 'B-Name': 'Name','B-Skills': 'Skills',
                'B-UNKNOWN': 'UNKNOWN','B-University': 'University', 'B-YOE': 'YOE', 'B-projects': 'Projects', 
                'B-state': 'State', 'B-training': 'Training', 'I-Achievements': 'Achievements',
                'I-Address': 'Address', 'I-CRT': 'CRT', 'I-Certifications': 'Certifications', 
                'I-College': 'College','I-Companies': 'Companies', 'I-Degree': 'Degree', 'I-Designation': 'Designation',
                'I-Email': 'Email', 'I-Graduation': 'Graduation', 'I-Links': 'Links', 'I-Location': 'Location','I-Name': 'Name',
                'I-Skills': 'Skills','I-UNKNOWN': 'UNKNOWN', 'I-University': 'University','I-YOE': 'YOE','I-projects': 'Projects',
                'I-training': 'Training', 'L-Achievements': 'Achievements', 'L-Address': 'Address',
                'L-CRT': 'CRT', 'L-Certifications': 'Certifications', 'L-College': 'College', 'L-Companies':'Companies', 
                'L-Degree': 'Degree','L-Designation': 'Designation','L-Email': 'Email','L-Graduation': 'Graduation','L-Links': 'Links',
                'L-Location': 'Location','L-Name':'Name','L-Skills': 'Skills', 'L-UNKNOWN': 'UNKNOWN',
                'L-University': 'University','L-YOE': 'YOE','L-projects': 'Projects','L-state': 'State',
                'L-training': 'Training', 'O': 'O','U-Address': 'Address','U-CRT': 'CRT','U-Certifications': 'Certifications',
                'U-College': 'College','U-Companies': 'Companies','U-Degree': 'Degree','U-Designation': 'Designation',
                'U-Email': 'Email','U-Graduation': 'Graduation','U-Links': 'Links','U-Location': 'Location','U-Name': 'Name',
                'U-Skills': 'Skills','U-UNKNOWN': 'UNKNOWN','U-YOE': 'YOE','U-links': 'Links','U-state': 'State',
                 'X': 'X', '[CLS]': '[CLS]','[SEP]': '[SEP]'}

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(tag2idx))
model.load_state_dict(torch.load('/content/drive/My Drive/ner_resume_2.bin'))

<All keys matched successfully>

In [0]:
model.eval()
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
      

In [0]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFSyntaxError
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal
import io

def extract_text_from_pdf(pdf_path):
    
    if not isinstance(pdf_path, io.BytesIO):
        with open(pdf_path, 'rb') as fh:
            try:
                for page in PDFPage.get_pages(
                                fh,
                                caching=True,
                                check_extractable=True
                ):
                    resource_manager = PDFResourceManager()
                    fake_file_handle = io.StringIO()
                    converter = TextConverter(
                        resource_manager,
                        fake_file_handle,
                        codec='utf-8',
                        laparams=LAParams()
                    )
                    page_interpreter = PDFPageInterpreter(
                        resource_manager,
                        converter
                    )
                    page_interpreter.process_page(page)

                    text = fake_file_handle.getvalue()
                    yield text

                    converter.close()
                    fake_file_handle.close()
            except PDFSyntaxError:
                return
    else:
        try:
            for page in PDFPage.get_pages(
                pdf_path,
                caching=True,
                check_extractable=True
            ):
                resource_manager = PDFResourceManager()
                fake_file_handle = io.StringIO()
                converter = TextConverter(
                    resource_manager,
                    fake_file_handle,
                    codec='utf-8',
                    laparams=LAParams()
                )
                page_interpreter = PDFPageInterpreter(
                    resource_manager,
                    converter
                )
                page_interpreter.process_page(page)

                text = fake_file_handle.getvalue()
                yield text

                converter.close()
                fake_file_handle.close()
        except PDFSyntaxError:
            return

def extract_text(file_path, extension):
    text = ''
    if extension == '.pdf':
        for page in extract_text_from_pdf(file_path):
            text += ' ' + page
    return text

In [0]:
class T(object):
  def __init__(self):
    self.ret={}
  def predict(self,text):
    sent=text.splitlines()
    sent=[st for st in sent if st !='' or st not in string.punctuation]

    words = [tokenizer.tokenize(snt) for snt in sent]
    sent = ["[CLS] " + sentence + " [SEP]" for sentence in sent]
    text_sent=[tokenizer.tokenize(snt) for snt in sent]
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in text_sent],
                          maxlen=512, dtype="long", truncating="post", padding="post")


    attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

    test_data = TensorDataset(torch.tensor(input_ids), torch.tensor(attention_masks))
    test_sampler = RandomSampler(test_data)
    test_dataloader = DataLoader(test_data, batch_size=1)


    res_tag=[]

    model.eval()
    ind=0
    with torch.no_grad():
      for batch in test_dataloader:
          batch = tuple(t.to(device) for t in batch)
          input_ids, input_mask = batch
        
          logits = model(input_ids, token_type_ids=None, attention_mask=input_mask)
          logits = logits.detach().cpu().numpy()
          logits = [list(p) for p in np.argmax(logits, axis=2)]
          input_mask = input_mask.to('cpu').numpy()
    
          for i,mask in enumerate(input_mask):
              temp_1 = [] # Real one
              for j, m in enumerate(mask):
                  if m:
                      if  idx2tag[logits[i][j]] != "[CLS]" and idx2tag[logits[i][j]] != "[SEP]" : 
                          temp_1.append(res_idx_tag[idx2tag[logits[i][j]]])
                  else:
                      break
              res_tag.append(temp_1)

        
          self.process([res_tag[ind]],words[ind])
          ind+=1
      
    return self.ret
  def process(self,res_tag,words):
    tmp,indx,ents='',0,[]
    while (indx < len(res_tag[0])):
       if res_tag[0][indx]!='X':
        tmp+=words[indx]
        ents.append(res_tag[0][indx])
        indx+=1
        while(indx<len(res_tag[0])):
            if res_tag[0][indx]=='X':
                tmp+=words[indx][2:]
                indx+=1
            else:
                break
        tmp+=' '
    res={}

    for ent,tm in zip(ents,tmp.split(' ')):
      if ent in res:
          tmpt=res[ent]
          tmpt+=' '+tm
          res[ent]=tmpt
      else:
          res[ent]=tm
        
    for key, value in res.items() :
      if key !="O":
        if key in self.ret:
          if key !='Name' and key !='Location' and key !='College':
            tmp=self.ret[key]
            if key not in tmp:
              tmp.append(value)
            self.ret[key]=tmp
        else:
          self.ret[key]=[value]

In [0]:
T().predict(text)

{'Achievements': ['Ranked 11st in Assiut Competitive Programming Contests in',
  'Ranked 19th place in ACM Sa3edy Competitions place in'],
 'College': ['Assiut university'],
 'Companies': ['iNetworks'],
 'Degree': ['Student'],
 'Designation': ['Computer Science', 'Data Scientist'],
 'Location': ['Cairo'],
 'Name': ['Ahmed Muhammad Sayed'],
 'Projects': ['.', '.', 'classification', 'by .', '.'],
 'Skills': ['Undergraduate',
  '-ots',
  'Twittelab',
  'CodeForces',
  'Problem Solving + years .',
  'Mathematics NLP',
  'Pytorch']}

In [0]:
text=extract_text('/content/drive/My Drive/FResume Ahmed Muhammad.pdf','.pdf')

In [0]:
ret

{'Achievements': ['Ranked 11st in Assiut Competitive Programming Contests in',
  'Ranked 19th place in ACM Sa3edy Competitions place in'],
 'College': ['Assiut university'],
 'Companies': ['iNetworks'],
 'Degree': ['Student'],
 'Designation': ['Computer Science', 'Data Scientist'],
 'Location': ['Cairo'],
 'Name': ['Ahmed Muhammad Sayed'],
 'Projects': ['.', '.', 'classification', 'by .', '.'],
 'Skills': ['Undergraduate',
  '-ots',
  'Twittelab',
  'CodeForces',
  'Problem Solving + years .',
  'Mathematics NLP',
  'Pytorch']}